# Loading The Chisel Library Into a Notebook

In [3]:
val path = System.getProperty("user.dir") + "/../../resource/chisel_deps.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/home/parkdongho/dev/Digital-System-Design-with-Chisel/01-Digital-System-Design/03-Sequential-Logic/../../resource/chisel_deps.sc"

In [4]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

# Counter

In [158]:
class MyCounter(width: Int) {
  val en    = Wire(Bool())
  lazy val value = Reg(SInt(width.W))
  val next   = Wire(SInt(width.W))

  val incEn = Wire(Bool())
  incEn := false.B
  
  val decEn = Wire(Bool())
  decEn := false.B

  next := 0.S
  en := incEn ^ decEn
    
  when (en){
    value := value + next 
  }

  
  // Control
  def inc(): Unit =  {
    incEn := true.B
    println("INC")
  }

  def dec(): Unit =  {
    decEn := true.B
    println("DEC")
  }
    
  def clear(): Unit = {
    
  }
  
  // Status
  def isMax(): Bool = {
    false.B
    println("MAX")
  }
  
  def isZero(): Bool = {
    false.B
    println("Zero")
  }
    
}

defined class MyCounter

In [81]:
class Counter private (r: Range, oldN: Option[Int] = None) extends AffectsChiselPrefix {
  require(r.length > 0, s"Counter range cannot be empty, got: $r")
  require(r.start >= 0 && r.end >= 0, s"Counter range must be positive, got: $r")

  private lazy val delta = math.abs(r.step)
  private lazy val width = math.max(log2Up(r.last + 1), log2Up(r.head + 1))

  /** Number of counts before the counter resets
    *
    * @note Only defined for ranges starting at zero with steps of size 1. Use [[range]] for other
    * use cases.
    */
  def n: Int = oldN match {
    case Some(x) => x
    case None    =>
      // Reasonable for typical ranges
      require(
        r.start == 0 && r.step == 1,
        s"Counter.n only defined on ranges starting at 0 with step == 1, got $r. " +
          "Use underlying range."
      )
      r.last + 1
  }

  /** Creates a counter with the specified number of steps.
    *
    * @param n number of steps before the counter resets
    */
  def this(n: Int) { this(0 until math.max(1, n), Some(n)) }

  /** The current value of the counter. */
  val value = if (r.length > 1) RegInit(r.head.U(width.W)) else WireInit(r.head.U)

  /** The range of the counter values. */
  def range: Range = r

  /** Increments the counter by a step.
    *
    * @note The incremented value is registered and will be visible on the next clock cycle
    * @return whether the counter will wrap on the next clock cycle
    */
  def inc(): Bool = {
    if (r.length > 1) {
      val wrap = value === r.last.U

      if (r.step > 0) {
        // Increasing range
        value := value + delta.U
      } else {
        // Decreasing range
        value := value - delta.U
      }

      // We only need to explicitly wrap counters that don't start at zero, or
      // end on a power of two. Otherwise we just let the counter overflow
      // naturally to avoid wasting an extra mux.
      if (!(r.head == 0 && isPow2(r.last + delta))) {
        when(wrap) { value := r.head.U }
      }

      wrap
    } else {
      true.B
    }
  }

  /** Resets the counter to its initial value */
  def reset(): Unit = {
    value := r.head.U
  }
}

cmd81.sc:1: not found: type AffectsChiselPrefix
class Counter private (r: Range, oldN: Option[Int] = None) extends AffectsChiselPrefix {
                                                                   ^Compilation Failed

: 

In [81]:
visualize(() => new MyCounter(8))

cmd81.sc:1: type mismatch;
 found   : cmd81.this.cmd80.MyCounter
 required: chisel3.RawModule
val res81 = visualize(() => new MyCounter(8))
                            ^Compilation Failed

: 

In [143]:
class helloworld extends Module{
  val io = IO(new Bundle{
    val inc = Input(Bool())
    val dec = Input(Bool())
    val out = Output(SInt(8.W))
  })
  
  val counter = new MyCounter(8)
  
  when(io.dec === true.B){
    counter.dec()
  }
    
  when(io.inc === true.B){
    counter.inc()
  }
  
  io.out := counter.value
}

defined class helloworld

In [144]:
visualize(() => new helloworld)
println(getVerilog(new helloworld))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<?xml-stylesheet href="styles.css" type="text/css"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (0)
 -->
<!-- Title: helloworld Pages: 1 -->
 
 
 helloworld 
 
 
 cluster_helloworld 
 
 helloworld 
 
<!-- cluster_helloworld_clock -->
 
 cluster_helloworld_clock 
 
 clock 
 
<!-- cluster_helloworld_reset -->
 
 cluster_helloworld_reset 
 
 reset 
 
<!-- cluster_helloworld_io_inc -->
 
 cluster_helloworld_io_inc 
 
 io_inc 
 
<!-- op_xor_77 -->
 
 op_xor_77 
 
 
 ● 
 
 xor 
 
 ● 
 
 ● 
 
<!-- cluster_helloworld_io_inc->op_xor_77 -->
 
 cluster_helloworld_io_inc->op_xor_77:in1 
 
 
 
<!-- mux_1958674128 -->
 
 mux_1958674128 
 
 
 1 
 
 a? 
 
 ● 
 
 ● 
 
<!-- cluster_helloworld_io_inc->mux_1958674128 -->
 
 cluster_helloworld_io_inc->mux_1958674128:select 
 
 
 
<!-- cluster_helloworld_io_dec -->
 
 cluster_helloworld_io_dec 
 
 io_dec 
 
<!-- cluster_helloworld_io_dec->op_xor_77 -->
 
 cluster_helloworld_io_dec->op_xor_77:in2 
 
 
 
<!-- mux_1795800850 -->
 
 mux_1795800850 
 
 
 -1 
 
 a? 
 
 ● 
 
 0 
 
<!-- cluster_helloworld_io_dec->mux_1795800850 -->
 
 cluster_helloworld_io_dec->mux_1795800850:select 
 
 
 
<!-- cluster_helloworld_io_out -->
 
 cluster_helloworld_io_out 
 
 io_out 
 
<!-- struct_cluster_helloworld_value -->
 
 struct_cluster_helloworld_value 
 
 
 ● 
 
 value 
 
 ● 
 
<!-- struct_cluster_helloworld_value->cluster_helloworld_io_out -->
 
 struct_cluster_helloworld_value:out->cluster_helloworld_io_out 
 
 
 
<!-- op_add_80 -->
 
 op_add_80 
 
 
 ● 
 
 add 
 
 ● 
 
 ● 
 
<!-- struct_cluster_helloworld_value->op_add_80 -->
 
 struct_cluster_helloworld_value:out->op_add_80:in1 
 
 
 
<!-- mux_1717080173 -->
 
 mux_1717080173 
 
 
 ● 
 
 a? 
 
 ● 
 
 ● 
 
<!-- struct_cluster_helloworld_value->mux_1717080173 -->
 
 struct_cluster_helloworld_value:out->mux_1717080173:in2 
 
 
 
<!-- cluster_helloworld_en -->
 
 cluster_helloworld_en 
 
 en 
 
<!-- cluster_helloworld_en->mux_1717080173 -->
 
 cluster_helloworld_en->mux_1717080173:select 
 
 
 
<!-- op_xor_77->cluster_helloworld_en -->
 
 op_xor_77:out->cluster_helloworld_en 
 
 
 
<!-- cluster_helloworld_next -->
 
 cluster_helloworld_next 
 
 next 
 
<!-- cluster_helloworld_next->op_add_80 -->
 
 cluster_helloworld_next->op_add_80:in2 
 
 
 
<!-- op_pad_78 -->
 
 op_pad_78 
 
 
 ● 
 
 pad 
 
 ● 
 
<!-- op_pad_78->cluster_helloworld_next -->
 
 op_pad_78:out->cluster_helloworld_next 
 
 
 
<!-- mux_1958674128->op_pad_78 -->
 
 mux_1958674128:out->op_pad_78:in1 
 
 
 
<!-- op_pad_79 -->
 
 op_pad_79 
 
 
 ● 
 
 pad 
 
 ● 
 
<!-- op_pad_79->mux_1958674128 -->
 
 op_pad_79:out->mux_1958674128:in2 
 
 
 
<!-- mux_1795800850->op_pad_79 -->
 
 mux_1795800850:out->op_pad_79:in1 
 
 
 
<!-- cluster_helloworld__value_T -->
 
 cluster_helloworld__value_T 
 
 _value_T 
 
<!-- op_tail_81 -->
 
 op_tail_81 
 
 
 ● 
 
 tail 
 
 ● 
 
 1 
 
<!-- cluster_helloworld__value_T->op_tail_81 -->
 
 cluster_helloworld__value_T->op_tail_81:in1 
 
 
 
<!-- op_add_80->cluster_helloworld__value_T -->
 
 op_add_80:out->cluster_helloworld__value_T 
 
 
 
<!-- cluster_helloworld__value_T_1 -->
 
 cluster_helloworld__value_T_1 
 
 _value_T_1 
 
<!-- op_asSInt_82 -->
 
 op_asSInt_82 
 
 
 ● 
 
 asSInt 
 
 ● 
 
<!-- cluster_helloworld__value_T_1->op_asSInt_82 -->
 
 cluster_helloworld__value_T_1->op_asSInt_82:in1 
 
 
 
<!-- op_tail_81->cluster_helloworld__value_T_1 -->
 
 op_tail_81:out->cluster_helloworld__value_T_1 
 
 
 
<!-- cluster_helloworld__value_T_2 -->
 
 cluster_helloworld__value_T_2 
 
 _value_T_2 
 
<!-- cluster_helloworld__value_T_2->mux_1717080173 -->
 
 cluster_helloworld__value_T_2->mux_1717080173:in1 
 
 
 
<!-- op_asSInt_82->cluster_helloworld__value_T_2 -->
 
 op_asSInt_82:out->cluster_helloworld__value_T_2 
 
 
 
<!-- mux_1717080173->struct_cluster_helloworld_value -->
 
 mux_1717080173:out->struct_cluster_helloworld_value:in

module helloworld(
  input        clock,
  input        reset,
  input        io_inc,
  input        io_dec,
  output [7:0] io_out
);
  reg [7:0] value;
  wire  en = io_inc ^ io_dec;
  wire  _GEN_2 = io_dec ? $signed(1'sh1) : $signed(1'sh0);
  wire [1:0] _GEN_4 = io_inc ? $signed(2'sh1) : $signed({2{_GEN_2}});
  wire [7:0] next = {{6{_GEN_4[1]}},_GEN_4};
  wire [7:0] _value_T_2 = $signed(value) + $signed(next);
  assign io_out = value;
  always @(posedge clock) begin
    if (en) begin
      value <= _value_T_2;
    end
  end
endmodule


In [157]:
class ShiftFIFO extends Module{
  val io = IO(new Bundle{
    val enq = Flipped(DecoupledIO(UInt(8.W)))
    val deq = DecoupledIO(UInt(8.W))
  })

  val readAddrCounter = new MyCounter(8)
  io.deq.bits := ShiftMem(io.enq.bits, readAddrCounter.value)
  
  io.enq.ready :=  readAddrCounter.isMax()
  io.deq.valid := ~readAddrCounter.isZero()

  when(io.enq.fire){
    readAddrCounter.inc()
  }

  when(io.deq.fire){
    readAddrCounter.dec()
  }

}

object ShiftFIFO{
  
}

cmd157.sc:8: not found: value ShiftMem
  io.deq.bits := ShiftMem(io.enq.bits, readAddrCounter.value)
                 ^cmd157.sc:10: value isMax is not a member of cmd157.this.cmd137.MyCounter
  io.enq.ready :=  readAddrCounter.isMax()
                                   ^cmd157.sc:11: value isZero is not a member of cmd157.this.cmd137.MyCounter
  io.deq.valid := ~readAddrCounter.isZero()
                                   ^Compilation Failed

: 

In [157]:
visualize(() => new FIFO)

: 